# Visual SLAM

1. Initialization: At first frame, initialize map with 3D points from stereo.
2. Tracking:
   - at frame i+1, match keypoints between i and i+1


In [ ]:
import os
import numpy as np
import cv2
from pathlib import Path
import matplotlib.pyplot as plt
from tqdm import tqdm
import torch

from lightglue import LightGlue, SuperPoint
from lightglue.utils import rbd

from lac.perception.vision import LightGlueMatcher
from lac.perception.depth import project_pixel_to_rover
from lac.utils.frames import apply_transform
from lac.utils.plotting import plot_3d_points, plot_surface, plot_poses, plot_path_3d
from lac.util import load_data, grayscale_to_3ch_tensor
from lac.params import LAC_BASE_PATH, DT

%load_ext autoreload
%autoreload 2

In [ ]:
data_path = Path(LAC_BASE_PATH) / "output/DataCollectionAgent/map1_preset0_stereo_lights1.0"
initial_pose, lander_pose, poses, imu_data, cam_config = load_data(data_path)
print(f"Num poses: {len(poses)}")

map = np.load(
    Path(LAC_BASE_PATH) / "data/heightmaps/competition/Moon_Map_01_preset_0.dat",
    allow_pickle=True,
)

In [ ]:
LAST_FRAME = len(poses)

left_imgs = {}
right_imgs = {}

for img_name in os.listdir(data_path / "FrontLeft"):
    if int(img_name.split(".")[0]) > LAST_FRAME:
        continue
    left_imgs[int(img_name.split(".")[0])] = cv2.imread(
        str(data_path / "FrontLeft" / img_name), cv2.IMREAD_GRAYSCALE
    )

for img_name in os.listdir(data_path / "FrontRight"):
    if int(img_name.split(".")[0]) > LAST_FRAME:
        continue
    right_imgs[int(img_name.split(".")[0])] = cv2.imread(
        str(data_path / "FrontRight" / img_name), cv2.IMREAD_GRAYSCALE
    )

assert len(left_imgs.keys()) == len(right_imgs.keys())
img_idxs = sorted(left_imgs.keys())

In [ ]:
fig = plot_surface(map)
fig = plot_poses(poses[::20], fig=fig)
fig.show()

## Stereo (PnP) VO


In [ ]:
from lac.localization.slam.visual_odometry import StereoVisualOdometry

In [ ]:
svo = StereoVisualOdometry(cam_config)

In [ ]:
start_idx = 80
svo.initialize(poses[start_idx], left_imgs[start_idx], right_imgs[start_idx])

In [ ]:
svo_poses = [poses[start_idx]]

end_idx = img_idxs[-1]
# end_idx = 2000

for idx in tqdm(np.arange(start_idx + 2, end_idx, 2)):
    svo.track(left_imgs[idx], right_imgs[idx])
    svo_poses.append(svo.rover_pose)

In [ ]:
fig = plot_poses(poses, no_axes=True, color="black", name="Ground Truth")
fig = plot_poses(svo_poses, no_axes=True, fig=fig, color="red", name="Stereo (PnP) VO")
fig.show()

In [ ]:
len(svo_poses), len(poses)

In [ ]:
fig.write_html("stereo_pnp_vo_full_spiral.html")

# SymForce


In [ ]:
import time
import symforce

try:
    symforce.set_epsilon_to_symbol()
except symforce.AlreadyUsedEpsilon:
    print("Already set symforce epsilon")
    pass

from lac.localization.factor_graph import FactorGraph
from lac.localization.slam.feature_tracker import FeatureTracker

In [ ]:
tracker = FeatureTracker(cam_config)
graph = FactorGraph()

START_FRAME = 80
# tracker.initialize(poses[START_FRAME], left_imgs[START_FRAME], right_imgs[START_FRAME])
tracker.initialize(initial_pose, left_imgs[START_FRAME], right_imgs[START_FRAME])

In [ ]:
N = 1000
UPDATE_RATE = 10
WINDOW_SIZE = 20

curr_pose = initial_pose
graph.add_pose(0, initial_pose)

# i is step which is 0 for initial and starts at 1 for the first run_step call
for i in tqdm(range(1, N)):
    if i < START_FRAME:
        graph.add_pose(i, poses[i])
    else:
        graph.add_pose(i, svo_poses[i - START_FRAME])

    # Add IMU factors
    if i > 1:
        graph.add_accel_factor(i, imu_data[i - 1][:3])
    graph.add_gyro_factor(i, imu_data[i - 1][3:])

    # Add vision factors
    if i % 2 == 0 and i > START_FRAME:
        tracker.track(left_imgs[i])

        for k in range(len(tracker.track_ids)):
            graph.add_reprojection_factor(
                i, tracker.prev_pts[k], tracker.world_points[k], tracker.track_ids[k]
            )

    # Sliding window optimization
    # if i % UPDATE_RATE == 0:
    #     result = graph.optimize(window=(i - UPDATE_RATE, i))
    #     curr_pose = graph.get_pose(i)

In [ ]:
# Batch optimization
result = graph.optimize(verbose=True)
fgo_poses = graph.get_all_poses()

In [ ]:
fig = plot_poses(poses[:N], no_axes=True, color="black", name="Ground truth")
fig = plot_poses(fgo_poses[:N], fig=fig, no_axes=True, color="green", name="FGO")
fig.update_layout(height=900, width=1600, scene_aspectmode="data")
fig.show()

# GTSAM


In [ ]:
import gtsam
from gtsam.symbol_shorthand import B, V, X, L

from lac.localization.gtsam_factor_graph import GtsamFactorGraph

## GTSAM IMU factors


In [ ]:
from gtsam import imuBias, noiseModel, PriorFactorConstantBias

In [ ]:
g = 1.622
IMU_PARAMS = gtsam.PreintegrationParams.MakeSharedU(g)
I = np.eye(3)
IMU_PARAMS.setAccelerometerCovariance(I * 0.2)
IMU_PARAMS.setGyroscopeCovariance(I * 0.2)
IMU_PARAMS.setIntegrationCovariance(I * 0.2)

In [ ]:
graph = gtsam.NonlinearFactorGraph()
initial_estimate = gtsam.Values()

pose_key = X(0)
pose_noise = gtsam.noiseModel.Diagonal.Sigmas(0.5 * np.ones(6))
pose_0 = gtsam.Pose3(poses[0])
graph.push_back(gtsam.PriorFactorPose3(pose_key, pose_0, pose_noise))
initial_estimate.insert(pose_key, pose_0)

# IMU prior
bias_key = B(0)
# bias_noise = gtsam.noiseModel.Isotropic.Sigma(6, 0.01)
# graph.push_back(gtsam.PriorFactorConstantBias(bias_key, gtsam.imuBias.ConstantBias(), bias_noise))
# initial_estimate.insert(bias_key, gtsam.imuBias.ConstantBias())
zero_bias = imuBias.ConstantBias(np.zeros(3), np.zeros(3))
graph.add(PriorFactorConstantBias(bias_key, zero_bias, noiseModel.Constrained.All(6)))
initial_estimate.insert(bias_key, zero_bias)

velocity_key = V(0)
velocity_noise = gtsam.noiseModel.Isotropic.Sigma(3, 0.2)
velocity_0 = np.array([0.0, 0.0, 0])
graph.push_back(gtsam.PriorFactorVector(velocity_key, velocity_0, velocity_noise))
initial_estimate.insert(velocity_key, velocity_0)

# Preintegrator
accum = gtsam.PreintegratedImuMeasurements(IMU_PARAMS)

n_frames = 1000

for i in range(1, n_frames):
    accum.integrateMeasurement(imu_data[i, :3], imu_data[i, 3:], DT)

    pose_key += 1
    DELTA = gtsam.Pose3(
        gtsam.Rot3.Rodrigues(0, 0, 0.1 * np.random.randn()),
        gtsam.Point3(0.5 * np.random.randn(), 0.5 * np.random.randn(), 0.5 * np.random.randn()),
    )
    # Initialize with noisy ground truth poses
    # initial_estimate.insert(pose_key, gtsam.Pose3(poses[i]).compose(DELTA))
    initial_estimate.insert(pose_key, gtsam.Pose3(poses[i]))

    velocity_key += 1
    initial_estimate.insert(velocity_key, np.array([0.0, 0.0, 0]))

    graph.add(
        gtsam.ImuFactor(pose_key - 1, velocity_key - 1, pose_key, velocity_key, bias_key, accum)
    )

    accum.resetIntegration()

In [ ]:
params = gtsam.LevenbergMarquardtParams()
# params.setMaxIterations(100)
# params.setlambdaUpperBound(1.e+6)
# params.setlambdaLowerBound(0.1)
# params.setDiagonalDamping(1000)
# params.setVerbosity('ERROR')
# params.setVerbosityLM('SUMMARY')
# params.setRelativeErrorTol(1.e-9)
# params.setAbsoluteErrorTol(1.e-9)

In [ ]:
optimizer = gtsam.LevenbergMarquardtOptimizer(graph, initial_estimate, params)
# result = optimizer.optimize()
for i in range(10):
    print(f"Iteration {i + 1}, Total Error: {graph.error(optimizer.values())}")
    optimizer.iterate()

In [ ]:
est_traj = np.array([result.atPose3(X(k)).translation() for k in range(n_frames)])

fig = plot_poses(poses[:n_frames], no_axes=True, color="black", name="Ground Truth")
fig = plot_path_3d(est_traj, fig=fig, color="red", name="GTSAM IMU")
fig.show()

## GTSAM projection factors


In [ ]:
import gtsam
from gtsam.symbol_shorthand import B, V, X, L
from gtsam import Cal3_S2, GenericProjectionFactorCal3_S2

In [ ]:
from gtsam import Stere

In [ ]:
g = 1.622
IMU_PARAMS = gtsam.PreintegrationParams.MakeSharedU(g)
I = np.eye(3)
IMU_PARAMS.setAccelerometerCovariance(I * 0.2)
IMU_PARAMS.setGyroscopeCovariance(I * 0.2)
IMU_PARAMS.setIntegrationCovariance(I * 0.2)

In [ ]:
graph = gtsam.NonlinearFactorGraph()
initial_estimate = gtsam.Values()

pose_key = X(0)
pose_noise = gtsam.noiseModel.Diagonal.Sigmas(0.01 * np.ones(6))
pose_0 = gtsam.Pose3(poses[0])
graph.push_back(gtsam.PriorFactorPose3(pose_key, pose_0, pose_noise))
initial_estimate.insert(pose_key, pose_0)

velocity_key = V(0)
velocity_noise = gtsam.noiseModel.Isotropic.Sigma(3, 0.1)
velocity_0 = np.zeros(3)
graph.push_back(gtsam.PriorFactorVector(velocity_key, velocity_0, velocity_noise))

# Preintegrator
accum = gtsam.PreintegratedImuMeasurements(IMU_PARAMS)

n_frames = 100

for i in range(0, n_frames):
    accum.integrateMeasurement(imu_data[i, :3], imu_data[i, 3:], params.DT)
    pose_key += 1
    DELTA = gtsam.Pose3(
        gtsam.Rot3.Rodrigues(0, 0, 0.1 * np.random.randn()),
        gtsam.Point3(4 * np.random.randn(), 4 * np.random.randn(), 4 * np.random.randn()),
    )